In [289]:
from amplpy import AMPL
import pandas as pd

In [322]:
rideshares = pd.read_csv('./rideshare_tg22.csv')
airports = ['LAX', 'ONT', 'SNA', 'BUR']
car_types = ['UberX', 'UberXL']

In [323]:
ez_rideshares = rideshares
ez_rideshares.rename(columns={'Departure Time (Pacific Daylight Time)': 'Flight depart'}, inplace=True)

In [ ]:
# this cell preprocess the earliest and latest airport arrival time preferences for students.

# convert hour:minutes to number of hours 
ez_rideshares['Earliest Airport Arrival Time'] = ez_rideshares['Earliest Airport Arrival Time'].str.split(':').apply(lambda x: int(x[0]) + int(x[1])/60)
ez_rideshares['Latest Airport Arrival Time'] = ez_rideshares['Latest Airport Arrival Time'].str.split(':').apply(lambda x: int(x[0]) + int(x[1])/60)
# Rounded to 2 decimal places
ez_rideshares['Earliest Airport Arrival Time'] = ez_rideshares['Earliest Airport Arrival Time'].round(2)
ez_rideshares['Latest Airport Arrival Time'] = ez_rideshares['Latest Airport Arrival Time'].round(2)


# create df for student earliest arrival time and student latest arrival time to the airport with student name as index

earliest_arrival_time = pd.DataFrame({ 'Name': ez_rideshares['Name'], 'Earliest Airport Arrival Time': ez_rideshares['Earliest Airport Arrival Time'] })
earliest_arrival_time.set_index('Name', inplace=True)

latest_arrival_time=pd.DataFrame({ 'Name': ez_rideshares['Name'], 'Latest Airport Arrival Time': ez_rideshares['Latest Airport Arrival Time'] })
latest_arrival_time.set_index('Name', inplace=True)


In [293]:
cost_data = {
    ('LAX', 'UberX'): 86, ('LAX', 'UberXL'): 125,
    ('ONT', 'UberX'): 42, ('ONT', 'UberXL'): 80,
    ('SNA', 'UberX'): 75, ('SNA', 'UberXL'): 130,
    ('BUR', 'UberX'): 105, ('BUR', 'UberXL'): 150
}

In [294]:
capacity_data = {
    'UberX': 4,
    'UberXL': 6
}

In [ ]:
# this cell preprocesses the variable travel times for LAX, ONT, BUR, SNA
# depending on the different hour of the day.

# it coalesces these travel times into one 2d dataframe with row corresponding
# to time interval, column corresponding to airport before converting this
# to a dictionary keyed by tuples that AMPL can handle. 

df1 = pd.read_csv('lax.csv')
df2 = pd.read_csv('ont.csv')
df3 = pd.read_csv('bur.csv')
df4 = pd.read_csv('sna.csv')

df5 = pd.concat([df1, df2, df3, df4], axis=1, join='inner')

# select only the "Travel Time" columns
travel_time_cols = [col for col in df5.columns if "Travel Time" in col]

# create a new DataFrame with only Travel Time columns
df_travel_times = df5[travel_time_cols].copy()

travel_time_dict = {}
for ind, row in df_travel_times.iterrows():
    for airport in airports:
        key = (ind/2+0.5, airport)
        value = row[f"Travel Time {airport}"].item()
        travel_time_dict[key] = value

print(travel_time_dict.items())
# Alternatively, using dictionary comprehension

df_travel_times

dict_items([((0.5, 'LAX'), 1.4605555555555556), ((0.5, 'ONT'), 0.3444444444444444), ((0.5, 'SNA'), 0.9844444444444445), ((0.5, 'BUR'), 0.935), ((1.0, 'LAX'), 1.5419444444444443), ((1.0, 'ONT'), 0.3277777777777778), ((1.0, 'SNA'), 1.0005555555555554), ((1.0, 'BUR'), 0.9686111111111112), ((1.5, 'LAX'), 1.4808333333333332), ((1.5, 'ONT'), 0.3497222222222222), ((1.5, 'SNA'), 1.0174999999999998), ((1.5, 'BUR'), 1.021111111111111), ((2.0, 'LAX'), 1.368611111111111), ((2.0, 'ONT'), 0.3552777777777777), ((2.0, 'SNA'), 0.8919444444444444), ((2.0, 'BUR'), 0.9652777777777778), ((2.5, 'LAX'), 1.2469444444444444), ((2.5, 'ONT'), 0.3116666666666666), ((2.5, 'SNA'), 0.7713888888888889), ((2.5, 'BUR'), 0.8880555555555555), ((3.0, 'LAX'), 1.1069444444444445), ((3.0, 'ONT'), 0.2836111111111111), ((3.0, 'SNA'), 0.7130555555555556), ((3.0, 'BUR'), 0.8052777777777779), ((3.5, 'LAX'), 1.0275), ((3.5, 'ONT'), 0.2638888888888889), ((3.5, 'SNA'), 0.6763888888888889), ((3.5, 'BUR'), 0.7513888888888889), ((4.0, 

,Travel Time LAX,Travel Time ONT,Travel Time BUR,Travel Time SNA
0,1.460556,0.344444,0.935000,0.984444
1,1.541944,0.327778,0.968611,1.000556
2,1.480833,0.349722,1.021111,1.017500
3,1.368611,0.355278,0.965278,0.891944
4,1.246944,0.311667,0.888056,0.771389
5,1.106944,0.283611,0.805278,0.713056
6,1.027500,0.263889,0.751389,0.676389
7,1.001111,0.250556,0.718333,0.655833
8,1.027222,0.235000,0.709722,0.648611
9,1.018333,0.236111,0.701667,0.646667


In [296]:
pricing_multiplier = [1.00, 1.02, 1.02, 1.03, 1.03, 0.99, 1.04, 1.03, 1.03, 1.08, 1.08, 1.14, 1.14, 1.19, 1.19, 1.20, 1.20, 1.30, 1.14, 1.10, 1.13, 1.12, 1.15, 1.17, 1.16, 1.17, 1.17, 1.20, 1.20, 1.19, 1.23, 1.22, 1.26, 1.24, 1.27, 1.25, 1.25, 1.18, 1.18, 1.19, 1.19, 1.17, 1.08, 1.12, 1.12, 1.07, 1.07, 1.00 ]

# Initialize time-indexed cost data with correct time keys
time_indexed_cost_data = {}

# Populate cost data for each time period
for (origin, service), base_cost in cost_data.items():
    for t, multiplier in enumerate(pricing_multiplier, start=1):
        # Convert time index to match P (e.g., 0.5, 1.0, 1.5, ...)
        time_key = t * 0.5
        time_indexed_cost_data[(origin, service, time_key)] = round(base_cost * multiplier, 2)

# Example output
print(time_indexed_cost_data)

{('LAX', 'UberX', 0.5): 86.0, ('LAX', 'UberX', 1.0): 87.72, ('LAX', 'UberX', 1.5): 87.72, ('LAX', 'UberX', 2.0): 88.58, ('LAX', 'UberX', 2.5): 88.58, ('LAX', 'UberX', 3.0): 85.14, ('LAX', 'UberX', 3.5): 89.44, ('LAX', 'UberX', 4.0): 88.58, ('LAX', 'UberX', 4.5): 88.58, ('LAX', 'UberX', 5.0): 92.88, ('LAX', 'UberX', 5.5): 92.88, ('LAX', 'UberX', 6.0): 98.04, ('LAX', 'UberX', 6.5): 98.04, ('LAX', 'UberX', 7.0): 102.34, ('LAX', 'UberX', 7.5): 102.34, ('LAX', 'UberX', 8.0): 103.2, ('LAX', 'UberX', 8.5): 103.2, ('LAX', 'UberX', 9.0): 111.8, ('LAX', 'UberX', 9.5): 98.04, ('LAX', 'UberX', 10.0): 94.6, ('LAX', 'UberX', 10.5): 97.18, ('LAX', 'UberX', 11.0): 96.32, ('LAX', 'UberX', 11.5): 98.9, ('LAX', 'UberX', 12.0): 100.62, ('LAX', 'UberX', 12.5): 99.76, ('LAX', 'UberX', 13.0): 100.62, ('LAX', 'UberX', 13.5): 100.62, ('LAX', 'UberX', 14.0): 103.2, ('LAX', 'UberX', 14.5): 103.2, ('LAX', 'UberX', 15.0): 102.34, ('LAX', 'UberX', 15.5): 105.78, ('LAX', 'UberX', 16.0): 104.92, ('LAX', 'UberX', 16.5

In [ ]:
ez_rideshares['Flight depart'] = pd.to_datetime(ez_rideshares['Flight depart'], format='%H:%M')
ez_rideshares['Flight times'] = ez_rideshares['Flight depart'].dt.hour + 0.5 * (ez_rideshares['Flight depart'].dt.minute//30)

# ez_rideshares now has earliest arrival time, latest arrival time, flight times columns

In [ ]:
ampl = AMPL()
ampl.eval(r"""
    set A;
    set S;
    set P = 0.5..24 by 0.5;  # time of arrival to airport
    set C;

    param cost{A, C, P};
    param capacity{C};
    param travelTime{P, A};
    param flightTime{S, A};
    param earliestArrivalTime{S}; # earliest arrival time for student s
    param latestArrivalTime{S}; # latest arrival time for student s

    var numCars{A, P, C} >= 0 integer; 
    var leaving{A, S, P} binary;  # 1 if student s leaves for airport a at time p, else 0
    var departureTime{A, S};

    minimize objective: sum{a in A, p in P, c in C} numCars[a, p, c]*cost[a, c, p];

    subject to notTooLateConstraint{a in A, s in S, p in P}:
        p - leaving[a, s, p]*(latestArrivalTime[s] - travelTime[p, a]) <= 100*(1 - leaving[a, s, p]);

    subject to notTooEarlyConstraint{a in A, s in S, p in P}:
        leaving[a, s, p]*(earliestArrivalTime[s] - travelTime[p, a]) - p <= 100*(1 - leaving[a, s, p]);
    
    subject to capacityConstraint{a in A, p in P}:
        sum{s in S} leaving[a, s, p] <= sum{c in C} numCars[a, p, c] * capacity[c];
    
    subject to onlyOneActiveY{s in S}:
        sum{a in A, p in P} leaving[a, s, p] = 1;

    subject to defineDepartureTime{a in A, s in S}:
        departureTime[a, s] = sum{p in P} leaving[a, s, p] * p;

    correctAirportConstraint{a in A, s in S}:
        sum{p in P} leaving[a, s, p] <= 100 * flightTime[s, a];
""")

In [299]:
# reset index on ez_rideshares to start at 0
ez_rideshares.reset_index(drop=True, inplace=True)

In [300]:
# set flight time data equal to Flight Times form ez_rideshares
# format: {(ez_rideshares['Name'][i], 'ONT') : ez_rideshares['Flight times'][i]}

raw_flight_time_data = {}
for i in range(len(ez_rideshares)):
    raw_flight_time_data[(ez_rideshares['Name'][i], ez_rideshares['Airport'][i])] = ez_rideshares['Flight times'][i]

In [ ]:
flight_time_data = {}
for s in ez_rideshares['Name']:
    for j in airports:
        if (s, j) in raw_flight_time_data:
            flight_time_data[(s, j)] = raw_flight_time_data[(s, j)]
        else:
            flight_time_data[(s, j)] = 0

raw_latest_arrival_data = {}
for i in range(len(ez_rideshares)):
    raw_latest_arrival_data[(ez_rideshares['Name'][i], ez_rideshares['Airport'][i])] = ez_rideshares['Latest Airport Arrival Time'][i]

In [ ]:
ampl.set['A'] = airports
ampl.set['S'] = ez_rideshares['Name']
ampl.set['C'] = car_types

ampl.getParameter("cost").setValues(time_indexed_cost_data)
ampl.getParameter("capacity").setValues(capacity_data)
ampl.getParameter("travelTime").setValues(travel_time_dict)
ampl.getParameter("flightTime").setValues(flight_time_data)
ampl.getParameter("earliestArrivalTime").setValues(earliest_arrival_time)
ampl.getParameter("latestArrivalTime").setValues(latest_arrival_time)



In [303]:
# run the model
ampl.solve(solver='gurobi')

Gurobi 11.0.3:Gurobi 11.0.3: optimal solution; objective 868.87
8 simplex iterations
1 branching node
absmipgap=1.13687e-13, relmipgap=0


In [304]:
# display the results
student_airport_times = {}
for student in ez_rideshares['Name']:
    for a in airports:
        x = ampl.getValue(f'departureTime["{a}", "{student}"]')
        if x != 0:
            student_airport_times[student] = (a, x)
student_airport_times

{'Nithya Yeluri': ('ONT', 5),
 'Nina Jobanputra': ('ONT', 11),
 'Aanya Pratapneni ': ('ONT', 11),
 'Will Sedo': ('ONT', 12),
 'Marcella Todd': ('ONT', 11),
 'Kaanthi Pandhigunta': ('ONT', 12),
 'Cevi Bainton': ('ONT', 14),
 'Ally Dye': ('ONT', 14),
 'Maya Maranto': ('ONT', 16.5),
 'Alec Vercruysse': ('ONT', 16.5),
 'Rebecca Preis': ('ONT', 17),
 'Susan Li': ('ONT', 17),
 'Tanvi Krishnan ': ('ONT', 19),
 'Jennifer Li': ('ONT', 19),
 'Allison Marten': ('ONT', 19),
 'Aldrin Feliciano': ('ONT', 19),
 'Julianna Schalkwyk': ('ONT', 21.5),
 'Charles Weismann': ('ONT', 3),
 'Katrina Nguyen': ('ONT', 4.5),
 'Bennet Matazzoni': ('ONT', 14),
 'Thaxter Shaw': ('LAX', 19),
 'Alicia Krasner': ('LAX', 11),
 'Laura Vairus': ('LAX', 13.5),
 'Ashley Kim': ('SNA', 18.5)}

In [305]:
for a in airports:
    for i in range(1, 49):
        for car in car_types:
            numCars = ampl.getValue(f"numCars['{a}', {i/2}, '{car}']")
            if numCars > 0:
                print(f'Time {i/2} has {numCars} {car} car(s) to {a}')

Time 11.0 has 1 UberX car(s) to LAX
Time 13.5 has 1 UberX car(s) to LAX
Time 19.0 has 1 UberX car(s) to LAX
Time 3.0 has 1 UberX car(s) to ONT
Time 4.5 has 1 UberX car(s) to ONT
Time 5.0 has 1 UberX car(s) to ONT
Time 11.0 has 1 UberX car(s) to ONT
Time 12.0 has 1 UberX car(s) to ONT
Time 14.0 has 1 UberX car(s) to ONT
Time 16.5 has 1 UberX car(s) to ONT
Time 17.0 has 1 UberX car(s) to ONT
Time 19.0 has 1 UberX car(s) to ONT
Time 21.5 has 1 UberX car(s) to ONT
Time 18.5 has 1 UberX car(s) to SNA
